In [1]:
import os
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt
from shutil import copyfile

In [2]:
# setup paths

ffhq_path = '/Users/max/Desktop/FFHQ_256/'
p_path = '/Users/max/Desktop/realZfake_p/'
save_path = '/Users/max/Desktop/realZfake_res/'

os.mkdir(save_path + 'reals')
os.mkdir(save_path + 'latents')
os.mkdir(save_path + 'fakes')

In [3]:
# merge result files A, B, C, D

latents_list = []
fakes_list = []
losses_list = []

parts = ['X', 'Y', 'Z'] # ['A', 'B', 'C', 'D']

counter = 1
start = time.time()

for part in parts:
    
    file_lat = open(p_path + 'lat' + part + '.p', 'rb')
    file_fak = open(p_path + 'fak' + part + '.p', 'rb')
    file_los = open(p_path + 'los' + part + '.p', 'rb')
    
    latents_list.append(pickle.load(file_lat))
    fakes_list.append(pickle.load(file_fak))
    losses_list.append(pickle.load(file_los))
    
    print('ETA:', str(np.round((len(parts)-counter)*(time.time()-start)/counter/60)), 'min left.')
    counter += 1
    
latents = np.vstack(latents_list)
fakes = np.vstack(fakes_list)
losses = np.hstack(losses_list)

ids = np.arange(len(losses))

ETA: 0.0 min left.
ETA: 0.0 min left.
ETA: 0.0 min left.


In [7]:
# DEBUGGING

latents = latents[:12,:,:]
losses = losses[:12]
ids = ids[:12]

In [8]:
# exclude examples with high inversion loss

loss_threshold = 0.25

ids_keep = ids[losses<loss_threshold]
print('Accepting', str(np.round(1000*len(ids_keep)/len(ids))/10), '% of examples.')

Accepting 66.7 % of examples.


In [9]:
# save to files

counter = 1
start = time.time()
stamp = time.time()

for i in ids_keep:
    
    # real
    copyfile(ffhq_path + str(i).zfill(5) + '.png', save_path + 'reals/' + str(i).zfill(5) + '.png')
    
    # latent
    np.savetxt(save_path + 'latents/' + str(i).zfill(5) + '.csv', latents[i], delimiter=',')
    
    # fake
    plt.imsave(save_path + 'fakes/' + str(i).zfill(5) + '.png', (fakes[i].transpose(1,2,0)+1)/2)
    
    if time.time() - stamp > 10:
        stamp = time.time()
        print('ETA:', str(np.round((len(ids_keep)-counter)*(time.time()-start)/counter/60)), 'min left.')
        
    counter += 1

In [11]:
# test: read latent code

i = 1
test_z = np.genfromtxt(save_path + 'latents/' + str(i).zfill(5) + '.csv', delimiter=',')
test_z.shape

(14, 512)